In [ ]:
!pip install pysentimiento

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from pysentimiento import create_analyzer
def sentiment_analysis(text):
  analyzer = create_analyzer(task="sentiment", lang="en")
  label, score = analyzer.predict(text).output, analyzer.predict(text).probas
  scr = score[label]
  match label:
        # pattern 1
        case 'POS':
            label = 'Positive'
        # pattern 2
        case 'NEG':
            label = 'Negative'
        # pattern 3
        case 'NEU':
            label = 'Neutral'
  return {'label':label, 'score': scr}

In [ ]:
print(sentiment_analysis("I hate you"))

{'label': 'Negative', 'score': 0.9806600213050842}


In [ ]:
from json.decoder import JSONDecodeError
import pandas as pd
import re
# Define the sentiment analysis function (stub function)


# Load the Excel file
file_path = "/content/drive/MyDrive/collab_code/Final Database/Tornado_data.xlsx"  # Change this path if needed
df = pd.read_excel(file_path)

# Function to preprocess text
def preprocess_text(text):
    # Remove links (delete the row if a link is found)
    if "http" in text or "www." in text:
        return None

    # Remove the pattern like [56] (number inside square brackets)
    text = re.sub(r"\[\d+\]", "", text).strip()

    return text

# Iterate through each row and process the 'Message' column
sentiments = []
scores = []
for index, row in df.iterrows():
    message = row["Message"]

    # Preprocess the message text
    processed_message = preprocess_text(message)

    # Skip the row if it contains a link (processed_message is None)
    if processed_message is None:
        df.drop(index, inplace=True)
        continue

    # Get the sentiment from the LLM function
    try:
      sentiment_score = sentiment_analysis(processed_message)
      sentiment = sentiment_score['label']
      score = sentiment_score['score']
    except JSONDecodeError:
      sentiment = "Undefined"
      score = 1.0
      continue

    # Append the sentiment to the list
    sentiments.append(sentiment)
    scores.append(score)

    # Update the sentiment in the DataFrame
    df.at[index, "Sentiment"] = sentiment
    df.at[index, "Score"] = score

# Save the updated DataFrame back to the Excel file
output_path = "/content/drive/MyDrive/collab_code/Final Database/twitter-sentiment-pl-base_Final.xlsx"
df.to_excel(output_path, index=False)

print(f"Processing complete. The updated file is saved at: {output_path}")


Processing complete. The updated file is saved at: /content/drive/MyDrive/collab_code/Final Database/twitter-sentiment-pl-base_Final.xlsx
